 <font size="6">Сверточные нейронные сети</font>

# Введение в сверточные нейронные сети


## Полносвязная нейронная сеть 
Fully-connected Neural Network (FCN). В современных статьях чаще используется термин Multilayer Perceptron (MLP).

На прошлом занятии мы рассмотрели следующий подход для классификации изображений:

1. Превращаем исходные данные в вектор. 

***Примечание***: при обработке цветного изображения размером $32\times32$ пикселя ($32\times32\times3$), размерность входного вектора будет равна $3072$. Однако, в общем случае модель получает на вход набор одноразмерных векторов (матрицу), потому при обработке одного изображения размер матрицы будет $3072\times1$.

2. Перемножаем матрицу данных с матрицей весов. Размер последней может быть, например, $100\times3072$. Где $3072$ - размер входного вектора, а $100$ - количество признаков, которое мы хотим получить. Результат обработки одного изображения будет иметь размер $100\times1$.

3. Поэлементно применяем к полученной матрице нелинейную функцию (функцию активации), например Sigmoid или ReLu. Размерность данных при этом не меняется ($100\times1$). В результате получаем вектор активаций или признаков.

4. Используем полученные признаки как входные данные для нового слоя. Количество весов слоя будет зависеть от размерности входной матрицы и того, что мы хотим получить на выходе. Если мы делаем классификатор на $10$ классов, то матрица весов должна иметь размерность $10\times100$, и на этом можно остановиться. Но в общем случае количество слоев может быть произвольным.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_1.png" width="600">


На изображении представлена описанная выше нейронная сеть, функцией активации в которой является ReLu. Добавление второго слоя позволило модели использовать более одного шаблона на класс. Можно убедиться в этом, обучив модель на датасете CIFAR-10 и визуализировав веса первого слоя модели.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/mlp-templates.jpg" width="700">

Засчёт создания нескольких шаблонов для каждого из классов, многослойные архитектуры в общем случае показывают более высокую, чем перцептроны, эффективность на задачах классификации изображений. Однако подход с использованием многослойного перцептрона также имеет свои недостатки. Чтобы обнаружить их, присмотримся к происходящему с данными при предобработке.

## Нарушение связей между соседними пикселями

Единственным этапом предобработки данных при использовании многослойного перцептрона является превращение изображения в вектор. Основной его проблемой является "потеря связи" между соседними пикселями, что показано на изображении ниже. В общем случае, значения из соседних пикселей могут оказаться на большом расстоянии внутри результирующего вектора. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_9.png" width="600">

Словом, после преобразования мы теряем информацию о взаимном расположении значений на исходном изображении - при подборе весов никак не используется информация о том, что какие-то значения были "близки" друг к другу на исходном изображении. При этом мы, люди, понимаем важность взаимосвязей между пикселями.  

К примеру, набор тёмных пикселей находящихся поблизости внутри изображения может сообщить нам, что там располагается некий тёмный объект. Если же мы преобразуем изображение в вектор и эти значения "разбросает" по нему без сохранения пространственной структуры, то новому наблюдателю будет очень сложно понять, что где-то на исходном изображении была тёмная зона.  

Словом, передача нейросети информации о взаимном расположении пикселей может быть полезна. 

## Свертка с фильтром

Для реализации идеи локальности при обработке данных используется операция свёртки. Свёртка требует тензор параметров - "ядро свёртки", также называемое фильтром. По сути своей, операция свёртки - это та же самая взвешенная сумма с добавлением свободного члена, используемая в полносвязных линейных слоях. Ключевое отличие между линейным слоем и свёрткой заключается в том, что линейный слой получает на вход всё изображение, тогда как свёртка - небольшие фрагменты изображения. То есть, если при обработке линейным слоем всё изображение сравнивалось с шаблонами, то теперь лишь небольшие фрагменты изображения сравниваются с шаблонами. 

К примеру, на GIF ниже можно увидеть, как фильтр размером $3\times3$ применяется к одноканальному изображению размером $5\times5$. Шаблон имеет форму x-образного креста. В правой части можно увидеть, насколько фрагмент изображения под фильтром совпадает с шаблоном внутри фильтра.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-2.gif" width="300">

In [ ]:
import torch

image = torch.Tensor([[1,1,1,0,0],
                      [0,1,1,1,0],
                      [0,0,1,1,1],
                      [0,0,1,1,0],
                      [0,1,1,0,0]])

kernel = torch.Tensor([[1, 0, 1],
                       [0, 1, 0],
                       [1, 0, 1]])

img_size = image.shape
kernel_size = kernel.shape

result_size = (3, 3)
answer = torch.zeros(result_size)

for i in range(0, result_size[0]):
    for j in range(0, result_size[1]):
        segment = image[i:i+kernel_size[0], 
                        j:j+kernel_size[1]]
        answer[i, j] = torch.sum(segment * kernel)

print(answer)

## Примеры 'hancrafted' фильтров

В течение долгого времени веса в фильтрах подбирались вручную. Этого было достаточно для простой обработки изображений. О примерах мы поговорим далее.

### Фильтр Гаусса

Данный фильтр является одним из наиболее известных и используется в компьютерном зрении для снижения уровня шума в изображении. Такое название он носит, поскольку веса внутри фильтра инициируются двумерной функцией Гаусса. Центр распределения находится в центре фильтра. 

Ниже показан пример использования фильтра Гаусса размером $5 \times 5$ для размытия изображения. Размер фильтров может варьироваться от задачи к задаче - то есть, в некоторых случаях использование фильтра $5 \times 5$ даёт слишком сильное размытие, из-за чего может быть использован фильтр меньшего размера: к примеру, $4 \times 4$ или $3 \times 3$.

In [ ]:
import torch
from torch import nn
from skimage import data
import matplotlib.pyplot as plt

cat = data.chelsea().mean(axis = 2).astype('int32')

def im_show(arr):
  plt.axis('off')
  plt.imshow(arr, cmap='gray', vmin=0, vmax=255)
  plt.show()


im_show(cat)

g_kernel5 = torch.tensor([[1,4,7,4,1],
                          [4,16,26,16,4],
                          [7,26,41,26,7],
                          [4,16,26,16,4],
                          [1,4,7,4,1]]
                          ,dtype=torch.float) / 273 # sum of weights

gaussian_filter = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=g_kernel5.shape, bias=False)
gaussian_filter.weight.data = g_kernel5.unsqueeze(0).unsqueeze(0) #gaussian_kernel
gaussian_filter.weight.requires_grad = False
input_data = torch.tensor(cat).view(1,1,cat.shape[0],cat.shape[1]).type(torch.float)
out = gaussian_filter(input_data)
im_show(out[0, 0].numpy())

### Фильтр Собеля

На заре компьютерного зрения, людям приходилось явно выполнять feature engeneering при обработке изображений. То есть, было необходимо извлекать важную информацию из изображений. Довольно важной информацией на изображении являются границы - они могут быть использованы для большого количества задач. К примеру, для поиска дорожной разметки на изображении. Как видно на изображении ниже, фильтры - в частности, фильтр Собеля - могут прекрасно справляться с поиском границ. С данным фильтром Вы ещё раз столкнётесь в ходе практической работы.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/sobel_example.jpg" width="800">


### $\color{brown}{\text{Дополнительная информация}}$

#### Детектор границ Canny

На самом деле, если мы будем применять упомянутый выше алгоритм без подготовки изображения и дополнительных операций, то не получим хорошее качество обнаружения границ. Поэтому существуют детекторы границ, дополняющие оператор Собеля либо иные операторы (в алгоритме ниже оператор Собеля используется на шаге 2).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/canny.png" width="850">

#### Детекторы углов

Другим примером созданных вручную экстракторов признаков являются всевозможные детекторы углов. Одним из самых известных среди них является детектор углов Харриса. Мы не будем на нем останавливаться, но Вы можете прочесть о нем самостоятельно.

https://en.wikipedia.org/wiki/Harris_Corner_Detector


Как Вы увидите далее, сверточные нейронные сети по факту автоматизируют процесс извлечения подобных признаков, высвобождая время исследователей и инженеров.

Перемещая различные ядра свертки по изображению, можно выполнять манипуляции с изображениями. Вот как работают многие инструменты, которые можно найти в программах для редактирования изображений, таких как Photoshop или Gimp:

# Сверточный слой нейросети

## Основные параметры свёртки

Как уже упоминалось ранее, по сути операция свёртки представляет из себя вычисление взвешенной суммы со свободным членом, что довольно сильно напоминает линейный слой с тем лишь отличием, что последний применяется ко всем данным, а не к их части. Тем не менее, подобно линейному слою, операцию свёртки можно встроить в нейросеть и, путём градиентного спуска, подбирать параметры свёртки. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/filter1.png" width="300">

In [ ]:
import torch
import torch.nn as nn

local_linear = nn.Linear(9, 1, bias = False)

image = torch.Tensor([[1,1,1,0,0],
                      [0,1,1,1,0],
                      [0,0,1,1,1],
                      [0,0,1,1,0],
                      [0,1,1,0,0]])

segment_size = (3, 3)
weights = torch.Tensor([[1, 0, 1],
                       [0, 1, 0],
                       [1, 0, 1]])

local_linear.weight = nn.Parameter(weights.reshape(-1))

result_size = (3, 3)
answer = torch.zeros(result_size)

for i in range(0, result_size[0]):
    for j in range(0, result_size[1]):
        segment = image[i:i+segment_size[0], 
                        j:j+segment_size[1]].reshape(-1)
        answer[i, j] = local_linear(segment)

print(answer)

### Обработка цветных/многоканальных изображений

Ранее речь шла об одноканальных изображениях, однако в реальной жизни мы чаще сталкиваемся с RGB изображениями, в которых более одного канала - сразу три. В общем случае, изображение имеет размер $C \times H\times W$. К примеру, изображения из датасета CIFAR-10 имеют размер $3\times32\times32$.  

При обработке подобных изображений, используется уже не матрица, а трёхмерный тензор. Ниже демонстрируется процесс обработки фрагмента трёхмерного изображения фильтром размерности $3\times3$:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/conv1.png" width="400">


Результатом свертки входного тензора будет карта активации с глубиной $1$, вне зависимости от количества его каналов. Важен тот факт, что входной тензор не обязательно должен формироваться из пикселей изображения, а может быть выходом предыдущего свёрточного слоя. То есть, в общем случае передаётся некое трёхмерное представление входных данных (для чёрно-белых/одноканальных изображений глубина равна $1$).

Во время прямого прохода фильтр перемещается по ширине и высоте входного представления и вычисляются скалярные произведения между значениями фильтра и подлежащими значениями входного представления. В процессе формируется двумерная карта активации, которая содержит результат применения данного фильтра к каждой из областей входного представления. Сама карта активации и соответствие фрагмента входного представления значению карты активации показаны на изображении выше, тогда как процесс перемещения ядра свёртки продемонстрирован ниже:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_15-1.png" width="200">

In [ ]:
image = torch.randn((3, 8, 8))
kernel = torch.randn((3, 3, 3))
kernel_size = kernel.shape

answer = torch.zeros(6, 6)

for i in range(answer.shape[0]):
  for j in range(answer.shape[1]):
    segment = image[:, 
                    i:i+kernel_size[0], 
                    j:j+kernel_size[1]]
    answer[i, j] = torch.sum(segment * kernel)

print(answer)

### Использование нескольких фильтров

Подобно тому, как в перцептроне использовались несколько выходных нейронов, чтобы передать информацию о сходстве входного вектора с различными шаблонами, можно использовать несколько фильтров при свёртке. То есть, выполнять свёртку несколько $(C_{out}$, по количеству фильтров) раз с разными фильтрами. В результате появится несколько карт активации, имеющих размер $1\times H_{out}\times W_{out}$. При объединении этих карт будет получен тензор размерности $C_{out}\times H_{out} \times W_{out}$, где $C_{out}$ - количество фильтров, а также количество "каналов" в полученном представлении. Полученное представление может быть без проблем передано для следующей операции свёртки. 

На изображении ниже продемонстрирован результат применения сверточного слоя, содержащего $5$ фильтров, к изображению из CIFAR-10. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/depth.png" width="400"> 

In [ ]:
conv = torch.nn.Conv2d(in_channels=3, # Number of input channels (3 for RGB images)
                       out_channels=64, # Number of filters/output channels
                       kernel_size=5)

repeat_conv = torch.nn.Conv2d(in_channels=64, # Number of input channels (3 for RGB images)
                              out_channels=128, # Number of filters/output channels
                              kernel_size=5)

dummy_input_32 = torch.randn((1,3,32,32))
out_32 = conv(dummy_input_32)
print(out_32.shape) # [1, 64, .., ..]

out_32_2 = repeat_conv(out_32)
print(out_32_2.shape) # [1, 128, .., ..]

dummy_input_100 = torch.randn((1,3,100,100))
out_100 = conv(dummy_input_100)
print(out_100.shape) # [1, 64, .., ..]

В свёрточном слое указаны параметры, необходимые для создания ядра свёртки - количество каналов во входном представлении `in_channels` и размер ядра свёртки `kernel_size`. Также в нём необходимо указывать количество используемых фильтров `out_channels`. Стоит отметить, что, в отличие от полносвязанного слоя, свёрточный слой не требует информации о количестве значений во входном представлении и может быть использован как для представлений $C_{in} \times 32 \times 32$, так и $C_{in} \times 100 \times 100$. Словом, представления могут иметь практически любой размер, главное чтобы он был больше размера ядра свёртки.

### Уменьшение размера представлений

Заметим, что при свёртке ширина $W_{out}$ и высота $H_{out}$ **карты активаций** будут отличаться от **пространственных размерностей** $W_{in}$ и $H_{in}$ исходного тензора. К примеру, при обработке трёхканального тензора размера $32 \times 32$ ядром размера $5 \times 5$, можно будет произвести лишь $27$ $(32 - 5)$ сдвигов по вертикали и столько же - по горизонтали. Но при этом размер итоговой карты активаий будет развен $28 \times 28$, поскольку первый ряд либо столбец можно получить без сдвигов по вертикали либо горизонтали, соответственно. При повторном применении фильтра, размер каждой из сторон уменьшится ещё на $4$. Общая формула нового значения $N'$ пространственной размерности $N$ при размере стороны фильтра $F$ вычисляется по следующей формуле: $N' = N - F + 1$.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_20.png" width="700">

In [ ]:
import numpy as np
# let's do it manually

A = np.arange(9).reshape(3,3) # our matrix
C = np.array([[-1, 0], [0, 1]]) # filter

res = [[np.sum(A[:2, :2] * C), np.sum(A[:2, 1:3] * C)],
       [np.sum(A[1:3, :2] * C), np.sum(A[1:3, 1:3] * C)]]

print("Input: \n",A)
print("Filter: \n",C)
print("Result: \n",res)

Заметим, что при уменьшении размера представлений пиксели, находящиеся около краёв, участвуют в значительно меньшем количестве свёрток, чем пиксели в середине, хотя информация в них не обязательно менее ценна, чем информация из центральных пикселей. К примеру, пиксель в верхнем левом углу представления вне зависимости от размера фильтра будет принимать участие лишь в одной свёртке и информация о нём будет сохранена лишь в верхнем левом углу нового представления. 

### Расширение (padding)  

Для борьбы с описанной выше проблемой применяется дополнение входного тензора (англ. padding). В ходе него ширина и высота тензора увеличиваются засчёт приписывания столбцов и строк с некими значениями. К примеру, на изображении ниже перед свёрткой ядром размера $3\times3$ был применён padding нулями.  

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_21.png" width="250">

На примере убедимся, что это позволит нам сохранить пространственные размерности тензоров.

In [ ]:
image = torch.randn((1, 1, 7, 7))
print("Original tensor:\n", image)

padded_image = torch.zeros((1, 1, 9, 9))
padded_image[:, :, 1:-1, 1:-1] += image
print("Padded image:\n", padded_image)

conv_3 = torch.nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=3)
conv_5 = torch.nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=5)

conved_3 = conv_3(image)
conved_5 = conv_5(image)

conved_pad_3 = conv_3(padded_image)
conved_pad_5 = conv_5(padded_image)

print('\n\nOriginal shape:', image.shape)
print('Conved by 3 shape:', conved_3.shape)
print('Conved by 5 shape:', conved_5.shape)

print('\n\nPadded shape:', padded_image.shape)
print('Pad conved by 3 shape:', conved_pad_3.shape)
print('Pad conved by 5 shape:', conved_pad_5.shape)

Заметим, что дополнение одним рядом и одним столбцом не является универсальным решением: для фильтра размером 5 размер выходного тензора всё равно отличается от входного. Если мы немного видоизменим полученную выше формулу (используя размер дополнения $P$), то получим следующую формулу: $N' = N + 2\cdot P - F + 1$.  
Видно, что чтобы пространствнные размеры не изменялись ($N' = N$), для разных размеров фильтра требуются разные размеры паддинга. В общем случае, для размера фильтра $F$, требуемый размер дополнения $P$: $\displaystyle P = \frac{F-1}{2}$. 

Также не всегда может быть полезно использование дополнения именно нулями. Из часто используемых видов паддинга можно выделить следующие:  
$$
\begin{equation*}
\begin{split}
\textbf{Constant padding (значением }X\textbf{): } &XX[ABC]XX \\
\textbf{Replicate padding: } &AA[ABC]CC \\
\textbf{Reflect padding: } &CB[ABC]BA \\
\textbf{Circular padding: } &BC[ABC]AB \\
\end{split}
\end{equation*}
$$

Важными терминами, связанными с дополнением, являются "same padding" - использование дополнения, при котором размер выходного тензора не отличается от размера входного тензора, а также "valid padding" - отсутствие дополнения (поскольку тогда "верно" изменяется размер).

Ниже можно увидеть пример обработки RGB изображения с same constant padding (с использованием 0):

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-3.gif" width="780">

In [ ]:
image = torch.randn((1, 1, 7, 7))

padded_image = torch.zeros((1, 1, 9, 9))
padded_image[:, :, 1:-1, 1:-1] += image

conv_3 = torch.nn.Conv2d(in_channels=1, 
                        out_channels=1, 
                        kernel_size=3)

conv_3_padded = torch.nn.Conv2d(in_channels=1, 
                                out_channels=1, 
                                kernel_size=3,
                                padding = 1)

conv_3_padded.weight = conv_3.weight
conv_3_padded.bias = conv_3.bias

print("Explicitly padded:\n", conv_3(padded_image))
print("Implicitly padded:\n", conv_3_padded(image))

## Использование свёрточных слоёв

### Свёрточный слой = свёртка + активация

Поскольку операция свертки является линейной, аналогично полносвязному слою, к выходу сверточного слоя применяется* функция активации (например, ReLu).

*Так как функция активации применяется к тензору поэлементно, не важно какую имено форму имеет тензор, а значит и какой слой находился передней ней: полносвязанный или сверточный. 

In [ ]:
net = torch.nn.Sequential(
                           nn.Conv2d(in_channels = 3,
                                     out_channels = 6,
                                     kernel_size = 3),
                           nn.ReLU(), # Activation doesn't depend on input shape
                           nn.Flatten(),
                           nn.Linear(5400,100), # H = 1 + (32 - 3) / 1 
                           nn.ReLU(),  # Activation doesn't depend on input shape
                           nn.Linear(100,10)
                           )
dummy_input = torch.randn((1,3,32,32))
out = net(dummy_input)
print(out)

Отметим, что внутри нейросети появляется новый слой - `nn.Flatten()`. Он необходим, поскольку полносвязанный слой принимает на вход набор векторов значений, тогда как свёрточный слой возвращает набор трёхмерных тензоров. `nn.Flatten()` необходим, чтобы преобразовать набор тензоров в набор векторов. Эту же операцию можно было бы выполнить с помощью метода `.view()`.

### Общая структура свёрточной нейронной сети

Нейросетевая модель из предыдущего примера позволяет в общем случае понять структуру свёрточных нейронных сетей: после некоторого количества свёрточных слоёв, извлекающих локальную пространственную информацию, идут полносвязанные слои (как минимум в количестве одного), сопоставляющие извлечённую информацию. 

Внутри свёрточных слоёв происходит следующий процесс: первые слои нейронных сетей имеют малые рецептивные поля, т.е. им соответствует малая площадь на исходном изображении. Такие нейроны могут активироваться лишь на некие простые шаблоны, по типу углов или освещённости. Активации уровнем выше уже имеют большие рецептивные поля, в результате чего в картах активации может сохраняться информация о более нетривиальных геометрических фигурах. С каждым слоем свёрточной нейронной сети рецептивные поля увеличиваются, увеличивается и сложность шаблонов, на которые может реагировать слой. В некоторых случаях, рецептивное поле нейронов может быть размером со всё исходное изображение. Пример можно увидеть на схеме ниже.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/r-field.png" width="700">

Если изначально рецептивное поле имело размер $1\times1$, то после свёртки фильтром $K\times K$, оно стало иметь размер $K \times K$. То есть, рецептивное поле каждого нейрона увеличилось на $K-1$ по каждому из направлений. Не сложно самостоятельно убедиться, что данная закономерность сохранится при дальнейшем применении фильтров того или иного размера.

Однако, при больших размерах изображений (к примеру, $1024\times1024$), нейронная сеть становится очень глубокой, что приводит к появлению большого количества параметров. Тем не менее, рецептивное поле каждого из нейронов растить нужно, что делается путём искусственного увеличения рецептивного поля каждого из элементов тензора. В принципе, это можно сделать двумя принципиально разными способами, о примере каждого из которых мы и поговорим:  
1. Доработать операцию свёртки, чтобы рецептивное поле росло быстрее
2. Добавить некую промежуточную операцию, которая бы увеличивала рецептивные поля

### Шаг свёртки (Stride)

Вариантом доработки свёртки с целью увеличения рецептивного поля нейрона является изменение шага фильтра. Ранее фильтры двигались с шагом 1 по горизонтали и 1 по вертикали, то есть зоны применения фильтров были сдвинуты друг относительно друга на 1 строку либо столбец. При увеличении шага, количество применений фильтра изменяется. Свёртка массива $5\times5$ фильтром размером $3\times3$ с шагом $2$ по вертикали и горизонтали.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/Convolution_arithmetic.gif" width="350">

Казалось бы, с учеличением шага $S$ рецептивное поле не выросло - как увеличивалось с $1$ до $K$, так и увеличивается. Однако обратим внимание на иное: если раньше размерность $N$ становилась $N - F + 1$, то теперь она станет $\displaystyle 1 + \frac{N-F}{S}$. В результате, если раньше следующий фильтр с размером $K'$ имел рецептивное поле в $\displaystyle N \cdot \frac{K'}{N'} = N \cdot \frac{K'}{N - F + 1}$, то теперь $\displaystyle N \cdot \frac{K'}{N'} = N \cdot \frac{K'}{1 + \frac{N-F}{S}}$. Понятно, что $\displaystyle \frac{K'}{N - F + 1} \leq \frac{K'}{1 + \frac{N-F}{S}}$, потому рецептивное поле каждого нейрона увеличивается.

При этом важно заметить, что в некоторых случаях часть данных может не попасть в свёртку. К примеру, при $N = 7,\, K = 3,\, S = 3$. В данном случае, $\displaystyle N' = 1 + \frac{7 - 3}{3} = 2\frac13.$ В подобных ситуациях часть изображения не захватывается, в чём мы можем убедиться на наглядном примере:

In [ ]:
# Create torch tensor 7x7
input_t = torch.tensor([[[[1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99],
                        [1,1,1,1,1,1,99]]]],dtype=torch.float)
print(input_t)
conv = torch.nn.Conv2d(in_channels=1, # Number of channels
                       out_channels=1, # Number of filters
                       kernel_size=3, 
                       stride = 3,
                       bias = False # Don't use bias
                       )
print(conv.weight.shape) # shape = [1,1,3,3] batch, channels, H, W
conv.weight  = torch.nn.Parameter(torch.ones((1,1,3,3))) # Replace randow weights to ones
activation = conv(input_t) 
print(activation)

### Уплотнение (Субдискретизация, Pooling)

Другим вариантом стремительного увеличения размера рецептивного поля является использование дополнительных слоёв, требующих меньшее количество вычислительных ресурсов. Слои субдискретизации прекрасно выполняют эту функцию - подобно свёртке, производится разбиение изображения на небольшие сегменты, внутри которых выполняются операции, не требующие использования обучаемых весов. Два популярных примера подобных операций: получение максимального значения (max pooling) и получение среднего значения (average pooling). 

Аналогично разбиению на сегменты при свёртке, pooling имеют два параметра: размер фильтра $F$ (то есть, каждого из сегментов) и шаг $S$ (stride). Аналогичным свёрткам, при применении пуллинга формула размера стороны будет $\displaystyle N' = 1+ \frac{N-F}{S}.$ 

Ниже приведён пример использования обоих пуллингов при обработке массива.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_25.png" width="600">

In [ ]:
input = torch.tensor([[[
                     [1,1,2,4],
                     [5,6,7,8],
                     [3,2,1,0],
                     [1,2,3,4],
                     ]]],dtype=torch.float)

max_pool = torch.nn.MaxPool2d(2, stride=2)
avg_pool = torch.nn.AvgPool2d(2, stride=2)

print("Max pooling:\n",max_pool(input))
print("Average pooling:\n",avg_pool(input))

**Важно отметить**, что, в отличие от свёрток, уплотнение выполняется по каждому из каналов отдельно, в результате чего количество каналов не меняется, в отличие от применения фильтра при свёртке. К примеру, ниже можно увидеть визуализация применения max pooling к одному из каналов тензора, имеющего $64$ канала. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_24_1.png" width="350">

### Свёртка фильтром $1\times1$

Оба упомянутых выше метода позволяют значительно снизить количество весов в модели, путём быстрого увеличения рецептивного поля нейронов, что даёт возможность не делать модель очень глубокой. Однако если мы посмотрим на количество весов в слое нейронной сети, то заметим, что проблема может быть не только в глубине модели. 

Вспомним, что по сути своей при свёртке значения сегмента пропускаются через перцептрон с $C_{out}$ выходами и $C_{in} \cdot F_1 \cdot F_2$ входами, где $F_1$ и $F_2$ обозначают размер фильтра модели. Итого оказывается, что модель имеет $C_{out} \cdot C_{in} \cdot F_1 \cdot F_2$ весов и $C_{out}$ свободных членов, что в сумме даёт нам $C_{out} \cdot (C_{in} \cdot F_1 \cdot F_2 + 1)$ параметров. Каждая из четырёх переменных оказывает значительное влияние на количество весов. Уменьшение $F_1$ и $F_2$ может привести к уменьшению рецептивных полей в свёрточных слоях выше, что повлечёт увеличение количества слоёв и, скорее всего, увеличение общей сложности модели. Уменьшение $C_{out}$ может привести к ухудшению обобщающей способности моделей. В связи с этим всем, был придуман трюк для уменьшения $C_{in}$ - свёртка фильтром $1\times1$.

Идея заключается в следующем: для каждого элемента в карте активаций $H_{in}\times W_{in}$ есть вектор размерностью $C_{in}$. Элементы этого вектора сообщают, насколько сильно рецептивное поле соответствует каждому из $C_{in}$ шаблонов. Возможно, стоит попробовать найти некие полезные комбинации данных "признаков" внутри каждого элемента представления. Для этого прекрасно подходит поэлементно применённый перцептрон с $C_{temp}$ выходами, также являющийся набором из $C_{temp}$ фильтров свёртки размером $C_{in}\times1\times1$. Ниже приведён пример применения такого фильтра с целью снижения количества каналов.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_24.png" width="500">

In [ ]:
conv = torch.nn.Conv2d(in_channels=64, # Number of input channels
                       out_channels=32, # Number of filters
                       kernel_size=1)

dummy_input = torch.randn((1,64,56,56))
out = conv(dummy_input)
print("Input shape",dummy_input.shape)
print("Shape after 1x1 conv",out.shape) # [1, 64, 56, 56] batch, C_out, H_out, W_out

Теперь давайте выведем формулу, которая покажет, насколько использование промежуточного свёрточного слоя с $C_{temp}$ слоями уменьшит количество параметров при свёртке с $C_{out}$ фильтрами $F_1\times F_2$.

**Количество параметров при наивном решении:**  
$Param_{naive} = C_{out} \cdot (C_{in} \cdot F_1 \cdot F_2 + 1) = C_{in} \cdot (C_{out} \cdot F_1 \cdot F_2) + C_{out}.$

**Количество параметров при решении с $1\times1$:**  
$Param_{1\times1} = C_{temp} \cdot (C_{in}\cdot1\cdot1 + 1) + C_{out} \cdot (C_{temp} \cdot F_1 \cdot F_2 + 1) = C_{temp} \cdot (1 + C_{in} + C_{out}\cdot F_1 \cdot F_2) + C_{out}.$

Не сложно заметить, что $\displaystyle \frac{Param_{1\times1}}{Param_{naive}} \approx \frac{C_{temp}}{C_{in}}.$

### $\color{brown}{\text{Дополнительная информация}}$

#### Сравнение свёрточного и полносвязанного слоев 
(вообще, нужно очень сильно допродумать, как это оптимально подать)

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/conv_layer.png" width="550">

Мы успели довольно подробно рассмотреть свёрточный слой и заметить большое количество его сходств с полносвязным слоем (перцептроном). Теперь давайте попробуем сравнить эти два слоя по требуемому количеству параметров и операций умножения.  

В первую очередь, давайте определимся с размерами входного и получаемого тензоров. Пусть на вход передаётся $C_{in}\times H_{in}\times W_{in}$. На выходе пусть будет $K$ нейронов для полносвзяного слоя и $C_{out}\times H_{out} \times W_{out}$ для свёрточного слоя (фильтр имеет размер $C_{in} \times F_1 \times F_2$). $F_1$ и $F_2$ - размер ядра свёртки по высоте и ширине, соответственно.

Для простоты расчётов давайте примем, что шаг фильтра равен $1$ как по горизонтали, так и по вертикали. В таком случае, $H_{out} = H_{in} - F_1 + 1$, а $W_{out} = W_{in} - F_2 + 1$.
  
##### Количество параметров:  
***Полносвязанный слой:***  
Данный слой требует по параметру (весу) для всех связей между входными и выходными нейронами, то есть $C_{in} \cdot H_{in} \cdot W_{in} \cdot K$. Помимо этого, каждый из выходных нейронов имеет свободный член, общее количество которых $K$. Итого, количество обучаемых параметров: $(C_{in} \cdot H_{in} \cdot W_{in} + 1) \cdot K$.

***Свёрточный слой:***  
Для свёрточного слоя параметры связаны лишь с ядрами свёртки - внутри каждого ядра находятся $C_{in} \cdot F_1 \cdot F_2$ параметров, общее их количество - $C_{out}$. Помимо этого, каждое из ядер имеет свой собственный свободный член, потому общее количество обучаемых параметров: $(C_{in} \cdot F_1 \cdot F_2 + 1) \cdot C_{out}$.

***Сравнение количества параметров:***  
Поскольку количество свободных членов мало относительно количества весов, мы опутим их в этих расчётах. 
$$Comp_{param} = \frac{C_{in} \cdot H_{in} \cdot W_{in} \cdot K}{C_{in} \cdot F_1 \cdot F_2 \cdot C_{out}} = \frac{H_{in} \cdot W_{in} \cdot K}{F_1 \cdot F_2 \cdot C_{out}}.$$  


##### Количество умножений: 
***Полносвязанный слой:***  
В данном слое каждый вес используется лишь один раз, в результате общее количество умножений - $C_{in} \cdot H_{in} \cdot W_{in} \cdot K$.  

***Свёрточный слой:***
Заметим, что в отличие от перцептрона, свёрточная нейронная сеть использует каждый вес несколько раз, а именно - при подсчёте каждого из элементов карты активации. Размер карты активаций: $H_{out} \times W_{out}$. В итоге оказывается, что операция умножения применяется $C_{in} \cdot F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}$.

***Сравнение количества умножений:***
$$Comp_{mult} = \frac{C_{in} \cdot H_{in} \cdot W_{in} \cdot K}{C_{in} \cdot F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}} = \frac{H_{in} \cdot W_{in} \cdot K}{F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}} = \frac{Comp_{param}}{H_{out} \cdot W_{out}}.$$

##### Выводы:

Заметим, что выигрыш по количеству параметров при использовании свёрточного слоя омрачается большим количеством операций перемножения. Это было проблемой в течение долгого времени, пока вычисление операции свёртки не перевели на видеокарты (Graphical Processing Unit). При выполнении свёртки одного сегмента не требуется информация о результатах свёртки на другом сегменте, потому чисто теоретически можно выполнять данные операции параллельно, с чем как раз прекрасно справляются видеокарты. 

#### Другие виды сверток


##### 1D

Свертка - это математическая операция, которая объединяет две функции в третью. Например, допустим, что у нас есть две данные функции, **е(т)** а также **г(т)** и мы заинтересованы в применении одного поверх другого и вычислении площади пересечения:**f (t) * g (t) = (f * g) (t)**

В этом случае мы применяем **г(т)**(называется ядром) более **е(т)** и изменение ответа **(Е* г) (т)** согласно пересечению над областью обеих функций. Эта концепция свертки является наиболее используемой техникой обработки сигналов, поэтому она также применяется в компьютерном зрении, что можно рассматривать как обработку сигнала от нескольких датчиков RGB. Интуиция в эффективности свертки заключается в их способности фильтровать заданные входные сигналы в объединенный, более полезный результат.

В конкретном случае изображений сигнал лучше понимается с точки зрения матриц, а не форм волны. Поэтому наши функциие **(т)а** также **г(т)** теперь станетизображение *(матрица)* а также Ядро *(матрица)* соответственно. 
Еще раз, мы можем применить свертку, сдвинув одну из матриц поверх другой:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-1.png" width="300">

То, что вы видите выше, это просто матричное умножение скользящего окна на изображении с ядром, а затем сложение суммы. Сила сверток в контексте Computer Vision заключается в том, что они являются отличными экстракторами функций для области датчиков RGB. При индивидуальном рассмотрении каждый пиксель (датчик RGB) бессмысленно понимать, что содержит изображение. Именно отношения друг с другом в пространстве придают образу истинный смысл. Это относится к тому, как вы читаете эту статью на своем компьютере, когда пиксели, представляющие символы, и ваш мозг сопоставляют черные пиксели друг с другом в пространстве, чтобы сформировать концепцию символов.

https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

In [ ]:
#torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)
print(output.shape)

##### 3D

Одномерная операция свёртки используется для данных, имеющих последовательную структуру - текстов, аудиозаписей, цифровых сигналов. Как правило, данную структуру можно представить в виде временного измерения.  
Двумерная операция свёртки, о которой мы сегодня много говорили, применяется для обработки данных, имеющих пространственную структуру - то есть, играют роль взаимные расположения как по вертикали, так и по горизонтали. (добавить пример не изображения, но каналы + пространственная размерность + время!)  
Трёхмерная версия свёртки используется, когда данные имеют три независимых "пространственных" компоненты. Простейшим примером являются видео: к двумерной структуре самих изображений добавляется координата времени.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/3d_conv.png" width="600">



In [ ]:
#torch.nn.Conv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

# With cubic kernels and same stride
m = nn.Conv3d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv3d(16, 33, (3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
input = torch.randn(20, 16, 10, 50, 100)
output = m(input)
print(output.shape)

#### Справочник по сверткам

https://arxiv.org/pdf/1603.07285v1.pdf

https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d


the best: 
https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215


## Сверточная сеть

Теперь мы можем более детально взглянуть на типичную архитектуру свёрточной нейронной сети. Как ранее уже обсуждалось, в первую очередь необходимо последовательностью свёрточных слоёв и уплотнений достичь того, чтобы каждый элемент карты активации имел большое рецептивное поле, а значит мог отвечать за большие и сложные шаблоны. Затем данные карты активаций выпрямляются в вектора и передаются в полносвязанные слои, последовательность которых, используя глобальную информацию, возвращает значение целевой переменной.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_27.png" width="800">

#### LeNet

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/lenet5.png" width="900">

Примером сети посторенной по такой архитектуре является LeNet.
Была разработана в 1989г [Яном Ле Куном](https://en.wikipedia.org/wiki/Yann_LeCun). Сеть имела 5 слоев с обучаемыми весами, из них 2 сверточных.

Применялась в США для распознавания рукописных чисел на почтовых конвертах до начала 2000г.

Ниже представлена ее реализация на Pytorch:

In [ ]:
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension after pool
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x): # Expected input size ..x1x32x32
        x = self.conv1(x) # 32 -> 28 because padding = 0
        x = F.sigmoid(x)
        x = F.avg_pool2d(x, (2, 2)) # 28 -> 14  Avg pooling over a (2, 2) window
        x = self.conv2(x) #   14 -> 10 ; 1  + ((N-F) / stride)
        x = F.sigmoid(x)
        x = F.avg_pool2d(x, 2) # 13->6 
        x = torch.flatten(x,1) # ..x16x5x5 -> ..x400   ; alternatively use nn.Flatten()
        x = F.sigmoid(self.fc1(x)) # 400 -> 120
        x = F.sigmoid(self.fc2(x)) # 120 -> 84
        x = self.fc3(x) # 84 -> 10
        return x


net = Net()

In [ ]:
print(net)

Как видно, данная модель рассчитанна на одноканальные изображения размером 32x32. Чтобы подавать на вход изображения иного размера - к примеру, $28\times28$ для MNIST, необходмо изменить лишь параметры полносвязанных слоёв, поскольку операции свёртки и пуллинга могут использоваться на любых представлениях, пока их размеры $\geq$ соответствующих размеров фильтров и потому их менять нет необходимости.

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

Процесс обучения свёрточной нейронной сети повторяет процесс обучения полносвязанной нейронной сети.

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss().cuda()

target = torch.tensor([1])  # a dummy target, for example
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)

loss = criterion(output, target)
loss.backward() # backprop
optimizer.step()    # Does the update

# Визуализация


Нам может быть интересно, на какую информацию обращает внимание модель в процессе работы - на какие визульные шаблоны реагирует, насколько они трактуемы нами. Чтобы ответить на все эти вопросы, можно визуализировать карты активаций и веса фильтров свёртки.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-5.png" width="1000">

## Визуализация весов 

Веса фильтров на первом слое легко визуализировать. И результат легко интерпретируется, так как у фильтров такое же количество каналов, как и у цветных изображений (3).

Ниже приведен пример того, как это можно сделать для обученной модели в Pytorch (Alexnet).

In [ ]:
from torchvision import models,utils
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (20,20)

alexnet = models.alexnet(pretrained=True)
weight_tensor = alexnet.features[0].weight.data
print(weight_tensor.shape)

img_grid = utils.make_grid(weight_tensor,pad_value=10)
plt.imshow(np.transpose(img_grid, (1, 2, 0)))

Примеры визуализации весов фильтров первых слоёв нескольких популярных моделей, обученных на ImageNet.

*Krizhevsky, "One weird trick for parallelizing convolutional neural networks", arXiv 2014 He et al, "Deep Residual Learning for Image Recognition", CVPR 2016 Huang et al, "Densely Connected Convolutional Networks", CVPR 2017*

Видно, что модели научились улавливать простые геометрические формы - края под разными углами, точки того или иного цвета... Тем не менее, фильтры AlexNet'а оказались настолько большими, что частично захватили не только простую локальную информацию - можно заметить, что некоторые из них ловят сложные градиенты или решётки - это вызвано их сравнительно большим размером.

## Визуализация фильтров промежуточных слоев

К сожалению, выполнить ту же операцию для фильтров на скрытых слоях едва ли представляется разумным - в отличие от приятного трёхканального вида фильтров, который легко можно трактовать и визуализировать, фильтры поздних слоёв имеют гораздо больше каналов, что делает их удобное отображение практически невозможным. Пожалуй, единственным вариантом отображения является поканальное отображение весов, которое довольно сложно трактовать, в чём можно убедиться, взглянув на пример ниже.

**Higher Layer: Visualize Filter**
<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-7.png" width="700">

## Feature extractor


Альтернативным способом понимания эффективности обучения сети является анализ карт активаций с последних слоёв нейронной сети. Поскольку в процессе работы нейросети создаются промежуточные представления, имеющие меньшую размерность, весь процесс представляет из себя проекирование данных в пространство меньшей размерности. Новое представление может быть использовано для решения различных задач не только с помощью линейных слоёв нейросети, но и с помощью других методов машинного обучения.

В следующих примерах мы будем использовать представление, полученное на последнем скрытом слое свёрточной нейронной сети.

**Последний слой**

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-8.png" width="700">

Чтобы убедиться в полезности полученных представлений, выполним поиск ближайшего соседа для изображения в исходном виде, пиксельном пространстве. В результате будут получены изображения, не всегда относящиеся к тому же классу, что и исходное изображение.

Если же мы используем предобученную свёрточную нейронную сеть для получения представлений изображений в более низкой размерности, и затем применим алгоритм поиска соседей, то "близко" друг к другу окажутся изображения, принадлежащие одному классу, даже если есть визуально более похожие изображения из иных классов - фон и прочие неважные части изображения имеют меньшее влияние на представление. Успешное извлечение признаков отображает успешное обучение модели. Один из вариантов выполнения кластеризации изображений - кластеризация промежуточных представлений свёрточных нейронных сетей.

**Последний слой: ближайшие соседи**
<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-9.jpg" width="900">

## Визуализация карт активаций

На последних слоях свёрточной нейронной сети размеры рецептивных полей нейронов становятся сравнимы с размером исходного изображения, потому при визуализации их карт активации становится понятно, какие нейроны реагируют на какие части изображений.

К примеру, на изображении ниже активация выделенного нейрона достигнута благодаря пикселям, примерно соответствующим расположению лица человека, потому можно предположить, что он научился находить лица на изображении. Более подробно об этом можно почитать в [данной статье](http://yosinski.com/media/papers/Yosinski__2015__ICML_DL__Understanding_Neural_Networks_Through_Deep_Visualization__.pdf).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-12.png" width="700">

## $\color{brown}{\text{Дополнительная информация}}$

**Maximally Activating Patches**

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-13.jpg" width="900">


*Выберите слой и канал; например, conv5 имеет размер 128 x 13 x 13, выберите канал 17/128;*

*Пропустите через сеть множество изображений, запишите значения выбранного канала;*

*Визуализируйте участки изображения, соответствующие максимальным активациям.*

Ещё один способ интерпретации нейронов выполнен следующим образом: через обученную сеть пропускается большое количество изображений, для каждого из которых записываются значения активаций на определённом канале определённого представления. К примеру, на изображении выше были взяты несколько каналов из `conv5` слоя и для каждого из изображений были записаны значения активаций. 

Затем для каждого из интересующих нас каналов производится поиск изображений, на которых активация данного канала была максимальной, после чего визуализируется рецептивное поле значений, максимально сильно активировавших тот или иной нейрон. 

Первый из визуализированных каналов реагирует на тёмную окружность на белом фоне, являющуюся глазом либо носом животного. Четвёртый канал реаширует на текст. 

Визуализации в нижней части изображения соответствуют объектам - их рецептивное поле ещё выше. Видно, что одни нейроны реагируют на машины, другие - на людей и так далее. Словом, модель выделяет довольно интерпретируемые объекты, чего и стоило ожидать.

### Активация слоев

Наиболее очевидный метод визуализации заключается в том, чтобы показать активации сети во время прямого прохода. Для сетей ReLU активации обычно начинает выглядеть относительно сгущенной и плотной, но по мере развития обучения активации обычно становится более редкой и локализованной. 

Единственная опасная ловушка, которую можно легко заметить с помощью этой визуализации, заключается в том, что некоторые карты активации могут быть равны нулю для многих различных входов. Это может указывать на мертвые фильтры и может быть симптомом высокой скорости обучения.

**Which Pixels Matter? Saliency via Occlusion**

Mask part of the image before feeding to CNN, NN, check how much predicted probabilities change

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-14.jpg" width="600">

### Окклюзирующие части изображения
Предположим, что CNN относит изображение к классу "собака". Как мы можем быть уверены, что он действительно улавливает собаку на изображении, а не контекстуальные сигналы с фона или другого объекта? Один из способов исследовать, из какой части изображения исходит некоторое классификационное предсказание, заключается в построении графика вероятности интересующего класса (например, класса собак) в зависимости от положения объекта окклюдера. 

То есть мы перебираем области изображения, устанавливаем патч изображения равным нулю и смотрим на вероятность класса. Мы можем визуализировать вероятность как 2-мерную тепловую карту. Этот подход был использован в визуализации и понимании сверточных сетей Мэтью Зейлера.

https://cs231n.github.io/understanding-cnn/

**Which Pixels Matter? Saliency via Occlusion**

Mask part of the image before feeding to CNN, check how much predicted probabilities change

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-15.jpg" width="1000">

### Обратное распространение для определения важности

***Примечание:*** подробнее ознакомиться с информацией об изложенных ниже методах можно в [данной статье](https://arxiv.org/pdf/1312.6034.pdf).

Ключевым элементом другого варианта определения важности различных частей на изображении является использование обратного распространения ошибки. В отличие от обычного обучения, где мы считаем неизменным входное изображение и пытаемся считать градиент по параметрам модели, в данном случае мы фиксируем или замораживаем значения весов нейросети и считаем градиент по входному изображению. 

Модуль градиента в данном случае будет показывать, сколь сильно изменение значения данного пикселя изменит предсказание метки класса в ту или иную сторону. Нам важен лишь сам факт сильного влияния, потому при определении важности стоит смотреть на **модули** значений частных производных по каждому из значений внутри изображений. В результате каждый пиксель будет представлен тремя неотрицательными числами и для простоты визуализации мы можем выбрать лишь максимальное значение для каждого из пикселей, в результате получая одноканальное изображение с важностью пикселей. 

**Which pixels matter? Saliency via Backprop**

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-17.png" width="700">

На следующем изображении можно увидеть примеры применения описанной выше последовательности действий к различным изображениям. Как видно, модель действительно обращает внимание на важные фрагменты изображения, практически игнорируя фон и прочую информацию с изображений, не относящуюся к классам напрямую. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-18.png" width="700">

После получения карт важности, можно попробовать выполнить сегментацию объектов на изображении: если мы предполагаем, что модель обучилась и "важными" для неё будут лишь осмысленные объекты, а не некий шум, то при использовании карт важности в качестве масок сегментации мы должны получать неплохие результаты, что можно увидеть на изображении ниже.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-19.png" width="700">

Аналогичную операцию можно выполнять, но считать градиенты не по конечным предсказаниям классов, а по активациям некого нейрона. То есть, мы смотрим, какие части рецептивного поля нейрона стоит изменять, чтобы значение активации возросло. 

Чтобы изображение получилось более аккуратным, стоит через каждую ReLU пропускать лишь положительные значения градиента, в тех нейронах, которые не были занулены в ходе прямого прохода сигнала.

**Intermediate Features via (guided) backprop**

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06-21.png" width="900">

Как и ранее, мы возьмём фрагменты изображения, которые максимально активировали те или иные нейроны в нашей нейросети. На них мы выполним ту же самую операцию, что и для целых изображений, за исключением использования максимального значения производной по каждому из пикселей - теперь мы представляем производные в виде RGB изображений. 

На примерах ниже видно, что использование градиентов помогает ещё лучше понять, на что же смотрит нейросеть в спорных, сложных для трактовки ситуациях - к примеру, в нижнем ряду на первом изображении.  

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-22.png" width="700">

Данный метод может быть использован на любом уровне представлений нейронной сети, в том числе и на самых высокоуровневых, способных замечать некие сложные формы и объекты. Видно, что некоторые нейроны научились обнаруживать практически любых собак на изображении. Другие обращают внимание на лицо и важные его части.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-23.png" width="700">

### Визуализация Модели класса

Далее разберем методику визуализации моделей классов, изученную с помощью сверточных нейронных сетей. Этот метод визуализации состоит в численном генерировании изображения, которое является репрезентативным для класса CNN. 

Пусть $S_c(I)$ - оценка класса c, вычисленная классификационным слоем CNN для изображения $I$. Мы хотели бы найти регуляризованное изображение $L_2$, таким, чтобы показатель $S_c$ высоким:

$$\arg \max_{I} S_c(I) - \lambda||I||_2^2$$

где $\lambda$ - параметр регуляризации. Локально-оптимальный показатель $I$ может быть найден методом обратного распространения. Этот процесс напрямую связан с обучением CNN, где обратное распространение используется для оптимизации весов по слоям. Разница в том, что в нашем случае оптимизация выполняется по отношению к входному изображению, а значения весов устанавливаются на те, которые были найдены на этапе обучения. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-24.png" width="700">

Мы инициализировали оптимизацию нулевым изображением (в нашем случае CNN была обучена на данных изображения с нулевым средним), а затем добавили к результату среднее изображение обучающего набора. После этого мы можем считать частые производные предсказаний по значениям изображения и выполнить обновление в сторону **увеличения** предсказанной вероятности класса.

*1. Initialize image to zeros*

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-25.png" width="1000">

*Repeat:*
*2. Forward image to compute current scores*

*3. Backprop to get gradient of neuron value with respect to image pixels*

*4. Make a small update to the image*

Стоит также отметить, что мы использовали (ненормализованные) оценки классов $S_c$, а не апостериоры классов, возвращаемые слоем soft-max:

$$P_c = \frac{\exp S_c}{\sum_c\exp S_c}.$$

Причина в том, что максимизация класса posterior может быть достигнута за счет минимизации баллов других классов. Таким образом, мы оптимизируем $S_c$ для того, чтобы убедиться в том, что оптимизация концентрируется только на рассматриваемом классе $S_c$. 

На двух изображениях ниже можно увидеть примеры сгенерированных изображений, максимизирующих предсказания по тем или иным классам. На изображениях ниже можно заметить очертания целевых объектов - к примеру, на сгенерированном для класса "хаски" изображении отчётливо виден силуэт собаки и характерные элементы окраски. Аналогиная ситуация наблюдается и для класса "гусь", но там видно уже несколько различных вариантов расположения гуся на изображении. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-28.png" width="700">

Данный процесс генерации изображений можно с лёгкостью улучшить - стоит лишь периодически использовать фильтр Гаусса (о котором мы сегодня говорили) для сглаживания изображения, а также занулять отрицательные значения пикселей и частных производных. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-29.png" width="700">

Поскольку при данном процессе вносится небольшая неопределённость, могут быть сгенерированы различные изображения в зависимости от того, когда была выполнен набор описанных выше. На изображениях ниже видны примеры работы дополненного алгоритма обработки. Как видно, на изображениях гораздо лучше читаются силуэты целевых объектов, а также окружающего их фона.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-31.png" width="700">

Тот же самый процесс может быть применён при максимизации значений внутри фильтров, таким образом чтобы было легче понять, на что активируется тот или иной нейрон. В примере ниже приведены примеры для разных свёрточных слоёв, начиная от второго - с малым рецептивным полем - и заканчивая пятым, имеющим сравнительно большое рецептивное поле.

**Visualizing CNN Features: Gradient Ascent**

Use the same approach to visualize intermediate features

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-32.png" width="700">

## Практика

### Pytorch. Tensor

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_28.png" width="700">

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html